In [19]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

url_peliculas = 'https://raw.githubusercontent.com/filippoelian01/ProyectoFinal_BasesPython/refs/heads/main/tmdb_5000_movies.csv' # ¡REEMPLAZAR ESTA URL!
df = pd.read_csv(url_peliculas)
print(f"1. Datos cargados. Filas, Columnas originales: {df.shape}")

df_clean = df[(df['revenue'] > 0) & (df['budget'] > 0)].copy()

revenue_median = df_clean['revenue'].median()

df_clean['Hit_Flop'] = (df_clean['revenue'] > revenue_median).astype(int)
print(f"2. Umbral de Hit/Flop (Mediana del Revenue): {revenue_median:,.0f}")
print(f"   Distribución del Target:\n{df_clean['Hit_Flop'].value_counts()}")

columns_to_drop = [
    'revenue', 'budget', 'id', 'homepage', 'status', 'tagline', 'Hit_Flop',
    'overview', 'keywords', 'genres', 'production_companies', 'production_countries',
    'spoken_languages', 'crew', 'cast', 'title', 'original_title'
]
X = df_clean.drop(columns=columns_to_drop, errors='ignore') 
y = df_clean['Hit_Flop']
print(f"   Columnas finales en X: {list(X.columns)}")

numerical_features = ['vote_count', 'vote_average', 'runtime', 'popularity']

categorical_features = ['original_language'] 

preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_features),
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features)
    ],
    remainder='drop', 
    sparse_threshold=0 
)

X_processed = preprocessor.fit_transform(X)
X_processed_df = pd.DataFrame(X_processed)
print(f"3. Dimensionalidad después de codificación: {X_processed_df.shape}")

selector = SelectKBest(score_func=f_classif, k=K) 
selector.fit(X_processed_df, y)

X_selected = selector.transform(X_processed_df)
print(f"4. Dimensionalidad final después de SelectKBest (k={K}): {X_selected.shape}")

X_train, X_test, y_train, y_test = train_test_split(
    X_selected, y, test_size=0.3, random_state=42
)
print(f"   Tamaño de entrenamiento: {X_train.shape}")
print(f"   Tamaño de prueba: {X_test.shape}")

modelo_clasificacion = LogisticRegression(solver='liblinear', random_state=42, max_iter=1000)

modelo_clasificacion.fit(X_train, y_train)

y_pred = modelo_clasificacion.predict(X_test)
print("5. Modelo entrenado y predicciones generadas.")

accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)

print("\n=============================================")
print("== RESULTADOS DEL MODELO DE CLASIFICACIÓN ==")
print("=============================================")
print(f"Accuracy (Exactitud, total de aciertos): {accuracy:.4f}")
print(f"Precision: {precision:.4f} (De lo que predije como Hit, ¿cuánto fue Hit?)")
print(f"Recall (Sensibilidad): {recall:.4f} (De todos los Hits reales, ¿cuántos encontré?)")
print(f"F1 Score (Equilibrio Precision y Recall): {f1:.4f}")
print("\nMatriz de Confusión:\n", conf_matrix)

1. Datos cargados. Filas, Columnas originales: (4803, 20)
2. Umbral de Hit/Flop (Mediana del Revenue): 55,184,721
   Distribución del Target:
Hit_Flop
0    1615
1    1614
Name: count, dtype: int64
   Columnas finales en X: ['original_language', 'popularity', 'release_date', 'runtime', 'vote_average', 'vote_count']
3. Dimensionalidad después de codificación: (3229, 31)
4. Dimensionalidad final después de SelectKBest (k=50): (3229, 31)
   Tamaño de entrenamiento: (2260, 31)
   Tamaño de prueba: (969, 31)
5. Modelo entrenado y predicciones generadas.

== RESULTADOS DEL MODELO DE CLASIFICACIÓN ==
Accuracy (Exactitud, total de aciertos): 0.7616
Precision: 0.8110 (De lo que predije como Hit, ¿cuánto fue Hit?)
Recall (Sensibilidad): 0.6904 (De todos los Hits reales, ¿cuántos encontré?)
F1 Score (Equilibrio Precision y Recall): 0.7459

Matriz de Confusión:
 [[399  79]
 [152 339]]


C:\Users\elian\anaconda3\Lib\site-packages\sklearn\feature_selection\_univariate_selection.py:783: UserWarning: k=50 is greater than n_features=31. All the features will be returned.
  warnings.warn(


## 📝 Conclusiones Finales del Proyecto (iv)

**Problema Resuelto:** Clasificación Binaria de Películas (Hit vs. Flop), utilizando la mediana del revenue como umbral.

**Método de Reducción de Dimensionalidad (SelectKBest):**
Se utilizó la técnica SelectKBest para reducir el número de características. Aunque inicialmente la codificación One-Hot generó muchas más columnas, la limpieza de variables complejas en el Paso 1 dejó un total de 31 features (como indica el 'UserWarning' en la salida). Se intentó seleccionar **K=50** features, pero se utilizaron las **31 disponibles**, lo que es adecuado para el modelo.

**Algoritmo Elegido (Regresión Logística):**
Se seleccionó la Regresión Logística por su capacidad para manejar la clasificación binaria y su buena interpretabilidad.

**Resultados Obtenidos (Validación con 30% del dataset):**
* **Accuracy (Exactitud):** **0.7616**
* **Precision:** **0.8110**
* **Recall (Sensibilidad):** **0.6904**
* **F1 Score:** **0.7459**

**Matriz de Confusión:**
* **399** (Verdaderos Negativos): Flops predichos correctamente.
* **339** (Verdaderos Positivos): Hits predichos correctamente.
* **79** (Falsos Positivos): Predijo que era Hit, pero fue Flop (Error tipo I).
* **152** (Falsos Negativos): Predijo que era Flop, pero fue Hit (Error tipo II).

**Interpretación de las Métricas:**

1.  **Exactitud (Accuracy):** Un **76.16%** de exactitud indica que el modelo clasifica correctamente en tres de cada cuatro casos.
2.  **Precision (0.8110):** Esta es una métrica alta. Significa que, cuando el modelo predice que una película será un **Hit**, tiene una confiabilidad de **81.1%**. Esto es excelente para la toma de decisiones, ya que minimiza la inversión en películas que se esperarían exitosas y no lo son.
3.  **Recall (0.6904):** El valor de Recall es más bajo. Esto significa que el modelo solo logró identificar el **69.04%** de todos los Hits reales disponibles en los datos de prueba.
4.  **F1 Score (0.7459):** El F1 Score, que equilibra Precision y Recall, es sólido (cercano a 0.75). Sugiere que el modelo es **útil y tiene valor predictivo**.

**Conclusión General:**
El modelo de Regresión Logística, utilizando las **31 características más relevantes** del dataset de películas, es capaz de predecir la probabilidad de que una película sea un **Hit** con una **alta precisión** (81.1%), aunque todavía tiene margen de mejora en la identificación de todos los Hits posibles (Recall del 69%). Este modelo es valioso para un estudio de producción que busca **minimizar el riesgo de invertir** en una película que se predice como exitosa.
